In [1]:
import requests
import pyjq
import numpy as np
import pickle
import operator

In [21]:
key = 'dP30AyQMBVXRtUVey7AsU4sZfQM7RpOX'
years = [2017, 2018]
months = np.arange(11) + 1

In [22]:
result_dict = {}
for year in years:
    for month in months:
        url = 'https://api.nytimes.com/svc/archive/v1/%d/%d.json?api-key='%(year, month)
        url += key
        results = requests.get(url)
        data = results.json()
        print('Get result from year: %d, month: %d'%(year, month))
        jq_query = f'.response .docs [] | {{the_snippet: .snippet, the_headline: .headline, the_keyword: .keywords}}'
        output = pyjq.all(jq_query, data)

        for i in range(len(output)):
            if output[i]['the_snippet'] is None:
                continue
            if output[i]['the_headline']['main'] is not None:
                val = output[i]['the_snippet'] + output[i]['the_headline']['main']
            else:
                val = output[i]['the_snippet']

            for out in output[i]['the_keyword']:
                if out['value'] in result_dict.keys():
                    result_dict[out['value']].append(val)
                else:
                    result_dict[out['value']] = [val]
        print("Added Values to the dic")

Get result from year: 2017, month: 1
Added Values to the dic
Get result from year: 2017, month: 2
Added Values to the dic
Get result from year: 2017, month: 3
Added Values to the dic
Get result from year: 2017, month: 4
Added Values to the dic
Get result from year: 2017, month: 5
Added Values to the dic
Get result from year: 2017, month: 6
Added Values to the dic
Get result from year: 2017, month: 7
Added Values to the dic
Get result from year: 2017, month: 8
Added Values to the dic
Get result from year: 2017, month: 9
Added Values to the dic
Get result from year: 2017, month: 10
Added Values to the dic
Get result from year: 2017, month: 11
Added Values to the dic
Get result from year: 2018, month: 1
Added Values to the dic
Get result from year: 2018, month: 2
Added Values to the dic
Get result from year: 2018, month: 3
Added Values to the dic
Get result from year: 2018, month: 4
Added Values to the dic
Get result from year: 2018, month: 5
Added Values to the dic
Get result from year: 

ScriptRuntimeError: Cannot iterate over null (null)

In [2]:
#pickle.dump(result_dict, open("nyt_archive_2017-18.p", "wb")) # save the results
data1 = pickle.load(open("data/nyt_archive_2016-75.p", "rb"))
data2 = pickle.load(open("data/nyt_archive_2019.p", "rb"))
data3 = pickle.load(open("data/nyt_archive_2017-18.p", "rb"))
data1.update(data2)
data1.update(data3)

In [3]:
# Check the top keywords:
length = {key: len(value) for key, value in data1.items()}
length = sorted(length.items(), key=operator.itemgetter(1), reverse=True)

In [11]:
freq = {}
for name, number in length[:100]:
    freq[name] = number

In [12]:
pickle.dump(freq, open("text_wordcloud.p", "wb"))

In [33]:
mapping = {
'United States Politics and Government' : 'Politics',
'Trump, Donald J' : ['Politics', 'Election'],
'Politics and Government' : 'Politics',
'Presidential Election of 2020' : 'Election',
'Books and Literature' : ['Art', 'Media'],
'Democratic Party' : ['Politics', 'Election'],
'Movies' : ['Art', 'Media'],
'New York City' : 'Economy',
'United States International Relations' : 'Foreign Policy',
'Television' : 'Media',
'Women and Girls' : 'Gender',
'Real Estate and Housing (Residential)' : 'Economy',
'China' : 'Foreign Policy',
'Art' : 'Art',
'United States' : 'Politics',
'Weddings and Engagements' : 'Other',
'Deaths (Obituaries)' : 'Other',
'Immigration and Emigration' : 'Immigration',
'Blacks' : ['Immigration', 'Politics'],
'Social Media' : 'Media',
'Computers and the Internet' : 'Media',
'Theater' : 'Media',
'Discrimination' : ['Politics', 'Gender'],
'House of Representatives' : 'Politics',
'United States Defense and Military Forces' : 'Foreign Policy',
'International Trade and World Market' : 'Economy',
'Russian Interference in 2016 US Elections and Ties to Trump Associates' : 'Foreign Policy',
'Demonstrations, Protests and Riots' : ['Politics', 'Violence'],
'Great Britain' : ['Foreign Policy', 'Economy'],
'Law and Legislation' : 'Politics',
'United States Economy' : 'Economy',
'Biden, Joseph R Jr' : 'Politics',
'Race and Ethnicity' : ['Politics', 'Gender'],
'Global Warming' : 'Climate',
'Murders, Attempted Murders and Homicides' : 'Violence',
'Travel and Vacations' : 'Other',
'Baseball' : 'Sport',
'Fashion and Apparel' : 'Fashion',
'Sex Crimes' : ['Gender', 'Violence'],
'Cooking and Cookbooks' : 'Other',
'Defense and Military Forces' : 'Foreign Policy',
'Terrorism' : 'Foreign Policy',
'Pop and Rock Music' : ['Art', 'Media'],
'Labor and Jobs' : 'Economy',
'News and News Media' : 'Media',
'Illegal Immigration' : ['Immigration', 'Foreign Policy'],
'Children and Childhood' : 'Other',
'Suits and Litigation (Civil)' : 'Politics',
'Trump-Ukraine Whistle-blower Complaint and Impeachment Inquiry' : 'Politics',
'Mueller, Robert S III' : 'Politics',
'Elections' : 'Election',
'California' : 'Economy',
'Regulation and Deregulation of Industry' : 'Economy',
'Restaurants' : 'Economy',
'Basketball' : 'Sport',
'Deaths (Fatalities)' : 'Violence',
'Music' : 'Art',
'Presidential Election of 2016' : 'Politics',
'Dancing' : 'Art',
'Economic Conditions and Trends' : 'Economy',
'Impeachment' : 'Politics',
'Homosexuality and Bisexuality' : 'Gender',
'Football' : 'Sport',
'Health Insurance and Managed Care' : ['Economy', 'Health'],
'Tennis' : 'Sport',
'Greenhouse Gas Emissions' : 'Climate',
'Soccer' : 'Sport',
'Sexual Harassment' : 'Gender',
'Documentary Films and Programs' : 'Art',
'Rap and Hip-Hop' : 'Art',
'#MeToo Movement' : 'Gender',
'Stocks and Bonds' : 'Economy',
'Wages and Salaries' : 'Economy',
'Environment' : 'Climate',
'Income Inequality' : 'Economy',
'Gun Control' : 'Violence',
 "Women's Rights" : 'Gender'}

In [34]:
lumped_data = {}
for key in data1.keys():
    if key in mapping:
        mapped_key = mapping[key]
    else:
        mapped_key = 'Other'
    if type(mapped_key)==list:
        for key_it in mapped_key:
            if key_it in lumped_data.keys():
                lumped_data[key_it].append(data1[key])
            else:
                lumped_data[key_it] = data1[key]
    else:
        if mapped_key in lumped_data.keys():
            lumped_data[mapped_key].append(data1[key])
        else:
            lumped_data[mapped_key] = data1[key] 

In [35]:
# Check the top keywords:
length = {key: len(value) for key, value in lumped_data.items()}
length = sorted(length.items(), key=operator.itemgetter(1), reverse=True)

In [37]:
for key in lumped_data.keys():
    file = open('data/%s.txt'%(key), 'w')
    for sentence in lumped_data[key]:
        if type(sentence) == list:
            sentence = sentence[0]
        file.write(sentence+'\n')
    file.close()

In [39]:
length

[('Other', 193203),
 ('Politics', 15734),
 ('Election', 15734),
 ('Media', 4468),
 ('Fashion', 2270),
 ('Foreign Policy', 2205),
 ('Immigration', 1739),
 ('Economy', 1629),
 ('Health', 1434),
 ('Art', 1235),
 ('Gender', 1068),
 ('Sport', 951),
 ('Violence', 766),
 ('Climate', 574)]